# lecture06_金融数据的相关性和线性回归分析
b站链接：https://www.bilibili.com/video/av20727707/?p=8

本节重点：
- 相关性分析（不同标的、指数之间）
- 线性回归分析（一元、多元）
- beta值计算（CAPM模型）

## 相关性分析
`相关性分析（correlation analysis）`是研究现象之间是否存在某种依存关系，并对具体有依存关系的现象探讨其相关方向以及相关程度，是研究随机变量之间的相关关系的一种统计方法。

$$R = \frac{\sum({X_i} - \overline{X})({Y_i} - \overline{Y})}{\sqrt{\sum({X_i} - \overline{X})^2 \sum({Y_i} - \overline{Y})^2}}$$
$$相关系数$$

`散点图`

### 数据选择：标的每日的对数收益率

In [ ]:
import numpy as np

start_date = '2017-01-01'
end_date = '2017-12-31'

def get_daily_return(api, symbol, start_date, end_date):
    # 查询日收盘价信息
    df, msg = api.daily(
        symbol=symbol,
        start_date=start_date,
        end_date=end_date,
        fields='close',
        freq='id',
        adjust_code='post'
    )
    
    df.set_index('trade_date', inplace=True)
    
    # 计算每日的对数收益率
    change = df['close'] / df['close'].shift(1)
    df['ret'] = np.log(change)
    
    # 删除第一个NA值
    df.dropna(inplace=True)
    return df['ret']

### 指数之间的相关性（1）
分析上证50与50ETF的相关性

In [ ]:


df_etf = get_daily_return(api, '500050.SH', start_date, end_date)
df_index = get_daily_return(api, '000016.SH', start_date, end_date)

np.corrcoef(df_etf, df_index)


import matplotlib.pyplot as plt

plt.scatter(df_etf, df_index)

# 输出相关系数矩阵

### 指数之间的相关性（2）
分析沪深300与300ETF的相关性

In [ ]:
df_etf = get_daily_return(api, '510300.SH', start_date, end_date)
df_index = get_daily_return(api, '000300.SH', start_date, end_date)

np.corrcoef(df_etf, df_index)


import matplotlib.pyplot as plt

plt.scatter(df_etf, df_index)

# 输出相关系数矩阵

### 指数之间的相关性（3）
分析上证50与沪深300的相关性

### 股票之间的相关性
分析贵州茅台（600519）与五粮液（000858）的相关性

## 一元线性回归
linear regression<br/>
$y = ax + b$<br>
search for $(a,b)$ to minimize estimation error

In [ ]:
import sicpy.stats
result = sicpy.stats.linregress(df_stock1, df_stock2)
print(result)

In [ ]:
x = np.arrange(-0.07, 0.07, 0.001)
y = x * result.slope + result.intercept  # result是线性回归的结果，slope斜率，intercept截距

plt.scatter(df_stock1, df_stock2)
plt.plot(x,y, color='r')

## 多元线性回归
$Y=\beta_0 + \beta_1X_1 + \beta_2X_2+......+\beta_mX_m+\varepsilon$

In [ ]:
# 沪深300，中证500，中证800
idx_hs500 = get_daily_return(api, '000300', start_date, end_date)
idx_zz500 = get_daily_return(api, '000905', start_date, end_date)
idx_zz800 = get_daily_return(api, '000906', start_date, end_date)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(idx_hs300, idx_zz500, idx_zz800)

### 最小二乘法OLS
F-Test：显著

参数： 显著

In [ ]:
import statsmodels.api as sm
import pandas as pd

In [ ]:
x =pd.concat([idx_hs300, idx_zz500, idx_zz800], axis=1)
x.columns = ['idx_hs300', 'idx_zz500']

x = sm.add_constant(x)
model = sm.OLS(idx_zz800,x)
results = model.fit()
results.summary()

### beta系数
贝塔系数（Beta Coefficient）是一种评估证券系统性风险的工具，用以度量一种证券或者一个投资证券组合相对总体市场的波动性。
$$\beta_a=\frac{Cov(r_a,r_m)}{\sigma^2_m}【公式法】$$
$$R=\alpha+\beta R_m+\epsilon【CAPM模型，回归法】$$


#### beta系数的计算
- beta系数的定义
- $return of stock - \beta*(return of market)+\alpha$
- CAPM model
- beta系数计算更适合用月度收益数据

In [ ]:
def get_monthly_return(api, symbol, start_date, end_date):
    # 查询月收盘价信息
    df, msg = api.daily(
        symbol=symbol,
        start_date=start_date,
        end_date=end_date,
        fields='close',
        freq-'1m',
        adjust_mode='post'
    )
    df.set_index('trade_date', inplace=True)
    
    # 计算每月的收益率
    df['ret'] = df['close'] / df['close'].shift(1)-1
    
    # 删除第一个NA值
    df.dropna(inplace=True)
return df['ret']

In [ ]:
# 用线性回归计算beta
def calc_beta(stock, index, start_date, end_date):
    df_stock = get_monthly_return(api, stock, start_date, end_date)
    df_index = get_monthly_return(api, index, start_date, end_date)
    
    # linear regression
    import scipy.stats
    result = scipy.stats.linregress(df_stock, df_index)
    print('beta of %s vs %s is %.3f' %(stock, index, result.slope))

使用线性回归计算beta的参数选择：
1. 使用月度收益率的数据；
2. 有研究表明，周期最好是4-6年；

#### 几个例子比较

In [ ]:
start_date = '2013-01-01'
end_date = '2017-12-31'

In [ ]:
# 贵州茅台相对于沪深300的beta （取一个周期内的）
calc_beta('600519', '000300', start_date, end_date)

In [ ]:
# 乐视网相对于上证50的beta
calc_beta('300104', '000016', start_date, end_date)